In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
data = pd.read_csv("CW_project4.txt", sep=" ", header=None)
data = data.drop(columns=6, axis=1)
data = data.rename(columns={7:6})


In [3]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(6,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

2023-09-01 11:46:44.352490: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2023-09-01 11:46:44.352517: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2023-09-01 11:46:44.352529: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2023-09-01 11:46:44.352625: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-01 11:46:44.352868: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


2023-09-01 11:46:50.844118: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 9530/20000 [=============>................] - ETA: 1:01 - loss: 106.3710 - accuracy: 0.7151

KeyboardInterrupt: 